# Postgres Demo notebook

At this stage the coco environment is configured & started.

We have also used a modified deployment, that uses a modified container which includes the Egeria Postgres connector.

We now need to configure the integration server & connector, and point to Postgres from where we will import metadata

## Setup

First we are going to setup some standard values that are normally setup elsewhere, but we'll repeat here
just in case & for clarity

In [24]:
# Standard checks for notebooks - ensure all are running, define standard functions
%run ../common/environment-check.ipynb


Checking OMAG Server Platform availability...
    Core Platform is active
        Checking OMAG Server cocoMDS2
           ... cocoMDS2 is configured
           ... cocoMDS2 is not known - needs to be started
                Starting server cocoMDS2 ...
                   ...cocoMDS2 failed to start
                       error message is: OMAG-ADMIN-500-004 The Asset Lineage OMAS service detected an unexpected java.lang.NullPointerException exception with message null during initialization
        Checking OMAG Server cocoMDS3
           ... cocoMDS3 is configured
           ... cocoMDS3 is active - ready to begin
        Checking OMAG Server cocoMDS5
           ... cocoMDS5 is configured
           ... cocoMDS5 is active - ready to begin
        Checking OMAG Server cocoMDS6
           ... cocoMDS6 is configured
           ... cocoMDS6 is active - ready to begin
    Data Lake Platform is active
        Checking OMAG Server cocoMDS1
           ... cocoMDS1 is configured
           ..

In [25]:
# language requirement
#import os

# admin operations - where/who
#adminUserId     = "garygeeke"
#devPlatformURL       = os.environ.get('devPlatformURL', 'https://localhost:9445')
#devPlatformName      = "Dev Platform"
#adminPlatformURL     = devPlatformURL
#adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'

# Event infrastructure & cohort membership
eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
jsonContentHeader = {'content-type':'application/json'}
eventBusBody      = {
    "producer": {
        "bootstrap.servers": eventBusURLroot
    },
    "consumer":{
        "bootstrap.servers": eventBusURLroot
    }
}
cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"

# Default metadata repository options
inMemoryRepositoryOption = "in-memory-repository"
graphRepositoryOption    = "local-graph-repository"

# Pick up which repo type to use from environment if set, otherwise default to inmemory 
metadataRepositoryType   = os.environ.get('repositoryType', inMemoryRepositoryOption)


## Setup a new metadata server

Here we will configure an additional metadata server

See https://egeria-project.org/services/omis/database-integrator/overview/ for more info on the required service to support database integration

In [26]:
# We already have an integration server - monitorDev01 -- but we'll use a new one for now

mdrServerName = 'dbisrv'
mdrServerPlatform = devPlatformURL

# Configure some of the basics (not all) including the required service to support database integration
configurePlatformURL(adminPlatformURL, adminUserId, mdrServerName, mdrServerPlatform)


configureAccessService(adminPlatformURL, adminUserId, mdrServerName, "data-manager", {})
configureMetadataRepository(adminPlatformURL, adminUserId, mdrServerName, metadataRepositoryType)

# We're going to join this server to the 'dev' cohort
configureEventBus(adminPlatformURL, adminUserId, mdrServerName, eventBusBody)
configureCohortMembership(adminPlatformURL, adminUserId, mdrServerName, devCohort)

# We'll now deploy the created configuration, and start the server
deployServerToPlatform(adminPlatformURL, adminUserId, mdrServerName, devPlatformURL)
activateServerOnPlatform(mdrServerName, "dev platform", devPlatformURL)



   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/server-url-root?url=https://lab-dev:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the data-manager access service for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/access-services/data-manager )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the metadata repository...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/local-repository/mode/in-memory-repository )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the event bus for this server...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/dbisrv/event-bus )
   ...... Response:  {'class': 'VoidRespon

True

In [27]:
# Setup and integration server

In [28]:
# We'll setup a new integration server on the dev platform
intServerName   = "intserv"
intServerPlatform=devPlatformURL

# Config information needed for the postgres connector
pgConnectorUser="pguser"
pgConnectorName="postgresql"
pgConnectorClass="org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider"
integrationServiceURLMarker="database-integrator"

# Authentication and URL for postgres - in a final lab this would be stored in secrets
pgUser="employees"
pgPass=",yDH[4|ETOp<1u*ai+0A{,,q"
pgURL="jdbc:postgresql://1	:5432/employees"

# Setup some of the basics
configurePlatformURL(adminPlatformURL, adminUserId, intServerName, intServerPlatform)
configureDefaultAuditLog(adminPlatformURL, adminUserId, intServerName)




   ... configuring the platform the server will run on...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/server-url-root?url=https://lab-dev:9443 )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}
   ... configuring the default audit log...
   ...... (POST https://lab-dev:9443/open-metadata/admin-services/users/garygeeke/servers/intserv/audit-log-destinations/default )
   ...... Response:  {'class': 'VoidResponse', 'relatedHTTPCode': 200}


In [29]:
# request body needed to setup the postgres connector
integrationServiceOptions={}
connectorConfigs = [
         {
        "class": "IntegrationConnectorConfig",
        "connectorName": "postgresql",
        "connectorUserId": pgConnectorUser,
        "connection":
        {
          "class": "Connection",
          "userId": pgUser,
          "clearPassword": pgPass,
          "connectorType":
          {
            "class": "ConnectorType",
            "connectorProviderClassName": pgConnectorClass
          },
          "endpoint":
          {
            "class": "Endpoint",
            "address": pgURL
          },
          "recognizedConfigurationProperties": [
            "url",
            "ssl"
          ],
          "configurationProperties":
          {
            "url": pgURL,
            "ssl": "false"
          }
        },
        "metadataSourceQualifiedName": pgConnectorName,
        "refreshTimeInterval": "3456",
        "usesBlockingCalls": "false",
        "permittedSynchronization": "FROM_THIRD_PARTY"
      }
    ]

# Setup the integration service, and then deploy & start the server
configureIntegrationService(adminPlatformURL, adminUserId, intServerName, mdrServerName, 
                                mdrServerPlatform, integrationServiceURLMarker, integrationServiceOptions, 
                                connectorConfigs)

deployServerToPlatform(adminPlatformURL, adminUserId, intServerName, devPlatformURL)

activateServerOnPlatform(intServerName, "dev platform", devPlatformURL)

   ... configuring the database-integrator integration service in intserv integration daemon ...
{'class': 'IntegrationServiceRequestBody', 'omagserverPlatformRootURL': 'https://lab-dev:9443', 'omagserverName': 'dbisrv', 'integrationServiceOptions': {}, 'integrationConnectorConfigs': [{'class': 'IntegrationConnectorConfig', 'connectorName': 'postgresql', 'connectorUserId': 'pguser', 'connection': {'class': 'Connection', 'userId': 'employees', 'clearPassword': ',yDH[4|ETOp<1u*ai+0A{,,q', 'connectorType': {'class': 'ConnectorType', 'connectorProviderClassName': 'org.odpi.openmetadata.adapters.connectors.integration.postgres.PostgresDatabaseProvider'}, 'endpoint': {'class': 'Endpoint', 'address': 'jdbc:postgresql://1\t:5432/employees'}, 'recognizedConfigurationProperties': ['url', 'ssl'], 'configurationProperties': {'url': 'jdbc:postgresql://1\t:5432/employees', 'ssl': 'false'}}, 'metadataSourceQualifiedName': 'postgresql', 'refreshTimeInterval': '3456', 'usesBlockingCalls': 'false', 'per

True